# Defining a little GAN for generating images

This is based on Francois Chollet's GAN example and uses a convolutional generator and discriminator.

In [1]:
## Little conv GAN design from Chollet 2017

import keras
from keras import layers
import numpy as np
from IPython.display import Image, display

# Data parameters
latent_dim = 32
height = 32
width = 32
channels = 3

Using TensorFlow backend.


In [2]:
# Download data from Charles' UiO webspace.
#!wget http://folk.uio.no/charlepm/datasets/gan/Xtrain.npy
#!wget http://folk.uio.no/charlepm/datasets/gan/ytrain.npy
  
X_train = np.load('./Xtrain.npy')
y_train = np.load('./ytrain.npy')

--2019-08-30 15:56:59--  http://folk.uio.no/charlepm/datasets/gan/Xtrain.npy
Resolving folk.uio.no (folk.uio.no)... 129.240.121.81
Connecting to folk.uio.no (folk.uio.no)|129.240.121.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22829216 (22M)
Saving to: ‘Xtrain.npy’

Xtrain.npy          100%[===================>]  21.77M  2.97MB/s    in 6.6s    

2019-08-30 15:57:06 (3.32 MB/s) - ‘Xtrain.npy’ saved [22829216/22829216]

--2019-08-30 15:57:06--  http://folk.uio.no/charlepm/datasets/gan/ytrain.npy
Resolving folk.uio.no (folk.uio.no)... 129.240.121.81
Connecting to folk.uio.no (folk.uio.no)|129.240.121.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22829216 (22M)
Saving to: ‘ytrain.npy’

ytrain.npy          100%[===================>]  21.77M  2.95MB/s    in 6.8s    

2019-08-30 15:57:13 (3.21 MB/s) - ‘ytrain.npy’ saved [22829216/22829216]



In [ ]:
# Get the training data (CIFAR-10)
import os
from keras.preprocessing import image
(x_train, y_train), (_,_) = keras.datasets.cifar10.load_data()
x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255
x_train.shape

In [ ]:
# Define the Generator Network.
generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128* 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 6, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding="same")(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

In [ ]:
# Define the discriminator newtork
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128,3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss="binary_crossentropy")


In [ ]:
# Define the complete adversarial network.
discriminator.trainable = False # set disc weights to be static (but just for this combined network)
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan= keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [ ]:
# Training Parameters
iterations = 100000
batch_size = 128
save_dir = '.'

In [ ]:
start = 0
for step in range(iterations):
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
  generated_images = generator.predict(random_latent_vectors)
  stop = start + batch_size
  real_images = x_train[start:stop]
  combined_images = np.concatenate([generated_images, real_images])
  labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
  labels += 0.05 * np.random.random(labels.shape)
  d_loss = discriminator.train_on_batch(combined_images, labels)
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
  misleading_targets = np.zeros((batch_size, 1))
  a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0
  if step % 100 == 0:
    gan.save_weights('gan.h5')
    print('step', step, '; disc loss', d_loss, '; ad loss', a_loss)

In [ ]:
# Generate a bunch of data
random_latent_vectors = np.random.normal(size=(16, latent_dim))
generated_images = generator.predict(random_latent_vectors)

for gen in generated_images:
  img = image.array_to_img(gen * 255., scale=False)
  display(img.resize((240, 240)))